In [10]:
#Ensures that the latest version of semantic-link libraries are loaded
%pip install -U semantic-link

%pip install -U semantic-link-labs

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 24, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [ ]:
###########################################################################################################
### Set Parameters
###########################################################################################################
#Define lakehouse and schema names
workspace_name = "<your workspace name>" #this is the workspace name where the notebook runs, and where the lakehouse lives
lakehouse_name = "<your lakehouse name>" #this is the name of the lakehouse, in the event that it needs to becreated by this notebook
schema_name = "<your schema name>"  #This is the name of the schema.  if using non-schema-enabled lakehouse, this MUST be dbo
semantic_model_name = "FabricInventory" ###NOTE: Defaults to FabricInventory, but it can be named anything. must be handled by the template file

#keyvault secret names: These need to be the names of the secrets in the given keyvault
key_vault_uri="<your keyvault url>"  #example: "https://fittool.vault.azure.net/"
tenant_id = "<your tenant secret name>" # the name of the secret that holds the tenant id
client_id = "<your secret name>" #the name of the secret that holds your service principal client id
client_secret = "<your secret name>" # the name of the secret that has the service principal secret value

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 26, Finished, Available, Finished)

In [12]:
#import necessary libraries
import sempy.fabric as fabric
#from sempy.fabric import set_service_principal
import pandas as pd
from datetime import date,datetime
import time
import sempy_labs as labs
from sempy_labs import admin, service_principal_authentication


# set default values used in future steps
full_lakehouse_name = f'{workspace_name}.{lakehouse_name}.{schema_name}'
current_date = date.today()
workspace_table_name = f'{full_lakehouse_name}.workspaces'
items_table_name = f'{full_lakehouse_name}.fabricitems'
capacities_table_name = f'{full_lakehouse_name}.capacities'

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 27, Finished, Available, Finished)

In [13]:
# Define variables and functions

###################################################################################################
### DEFINE FUNCTIONS
###################################################################################################
def logActivity(table_name,message):
    logquery = f'INSERT INTO {full_lakehouse_name}.inventorylog (timeval,category,Message) VALUES (CURRENT_TIMESTAMP(),"{table_name}","{message}")'
    #print(f'log query={logquery}')
    #print(logquery)
    spark.sql(logquery)



StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 28, Finished, Available, Finished)

In [14]:
## This cell gets all the workspaces, and loads them into the workspaces table

print("Starting Workspace table load....")
logActivity("Workspaces","Starting workspace table load")
#print(key_vault_uri)

with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):
    all_workspaces_df = admin.list_workspaces()
    #display(all_workspaces_df)
    

#match names to table names for simplicity sake
workspaces_df = all_workspaces_df[['Name','Id','State','domainId','Capacity Id','Type']]
workspaces_df = workspaces_df.rename(columns={'State':'state'})
workspaces_df = workspaces_df.rename(columns={'Name':'workspace_name'})
workspaces_df = workspaces_df.rename(columns={'Id':'workspace_id'})
workspaces_df = workspaces_df.rename(columns={'domainId':'domain_id'})
workspaces_df = workspaces_df.rename(columns={'Capacity Id':'capacity_id'})
workspaces_df = workspaces_df.rename(columns={'Type':'type'})
workspaces_df['type'] = workspaces_df['type'].str.replace('Workspace','Shared')
#add the timestamp column to the dataframe
workspaces_df['timeval'] = current_date
#display(workspaces_df)

#delete the current date entries, so only one entry per day should exist
delete_workspace_qeuery = f"DELETE FROM {workspace_table_name} WHERE timeval = '{current_date}'"
print("Deleting workspace entries for the current date....")
logActivity("Workspaces","Deleting workspace entries for the current date...")
spark.sql(delete_workspace_qeuery)
print("current date Workspace entries deleted.")
logActivity("Workspaces","current date workspace entries deleted.")

#insert rows into workspace table
spark_df = spark.createDataFrame(workspaces_df)
spark_df.write.mode("append").format("delta").saveAsTable(workspace_table_name)

print("Workspaces inserted")
logActivity("Workspaces","Workspaces inserted")

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 29, Finished, Available, Finished)

Starting Workspace table load....
Deleting workspace entries for the current date....
current date Workspace entries deleted.
Workspaces inserted


In [15]:
#get workspace items
print("Starting fabricItems....")
logActivity("FabricItems","Starting fabricItems")

print("Deleting current date fabric item entries....")
logActivity("FabricItems","Deleting current date fabric item entriess")
del_items_query = f"DELETE FROM {items_table_name} WHERE timeval = '{current_date}'"
spark.sql(del_items_query)
print("current date item entries deleted.")
logActivity("FabricItems","current date item entries deleted.")


with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

        items_df = admin.list_items()

#rename and clean up dataframe
items_df = items_df.rename(columns={'Capacity Id':'capacity_id'})
items_df = items_df.rename(columns={'Workspace Id':'workspace_id'})
items_df = items_df.rename(columns={'Creator Principal Type':'principal_type'})
items_df = items_df.rename(columns={'Creator Principal Display Name':'principal_name'})
items_df = items_df.rename(columns={'Last Updated Date':'last_updated'})
items_df = items_df.rename(columns={'State':'state'})
items_df = items_df.rename(columns={'Type':'item_type'})
items_df = items_df.rename(columns={'Item Name':'item_name'})
items_df = items_df.rename(columns={'Item Id':'item_id'})
items_df['last_updated'] = items_df['last_updated'].str.replace('T',' ')
del items_df['Creator User Principal Name']
del items_df['Creator Principal Id']
items_df['timeval'] = current_date
#display(items_df)
new_order = ["timeval", "item_name", "item_id","Description","item_type","workspace_id","capacity_id","principal_type","principal_name","last_updated","state"]

#reorder columns
final_items_df = items_df[new_order]
#display(final_items_df)
spark_df = spark.createDataFrame(final_items_df)

#display(spark_df)
#TEMPORARY TEST
#items_table_name = f'{full_lakehouse_name}.fabricitemstest'
spark_df.write.mode("append").format("delta").saveAsTable(items_table_name)
print(f'fabric items entered')
logActivity('FabricItems',f'fabric items entered')

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 30, Finished, Available, Finished)

Starting fabricItems....
Deleting current date fabric item entries....
current date item entries deleted.
fabric items entered


In [16]:
#get capacities items
print("Starting capacities....")
logActivity("Capacity","Starting capacities...")

print("Deleting current date capacity entries....")
logActivity("Capacity","Deleting current date capacity entriess")
del_items_query = f"DELETE FROM {capacities_table_name} WHERE timeval = '{current_date}'"
spark.sql(del_items_query)
print("current date capacity entries deleted.")
logActivity("Capacity","current date capacity entries deleted.")


with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

        capacities_df = admin.list_capacities()

#match names to table
capacities_df = capacities_df.rename(columns={'Capacity Id':'capacity_id'})
capacities_df = capacities_df.rename(columns={'Capacity Name':'capacity_name'})
capacities_df = capacities_df.rename(columns={'Sku':'sku'})
capacities_df = capacities_df.rename(columns={'Region':'region'})
capacities_df = capacities_df.rename(columns={'State':'state'})
capacities_df = capacities_df.rename(columns={'Admins':'admins'})
capacities_df['timeval'] = current_date
#capacities_df['admins'] = capacities_df['admins'].str.replace('[]','')
del capacities_df['Users']
del capacities_df['admins']
display(capacities_df)

spark_df = spark.createDataFrame(capacities_df)


spark_df.write.mode("append").format("delta").saveAsTable(capacities_table_name)
print(f'Capacities entered')
logActivity('Capacity',f'Capacities entered')

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 31, Finished, Available, Finished)

Starting capacities....
Deleting current date capacity entries....
current date capacity entries deleted.


SynapseWidget(Synapse.DataFrame, 634c8103-90a3-4d07-acae-00440b01c069)

Capacities entered


In [17]:

#Create date table
current_year = datetime.now().year
end_year = current_year + 1
#print(end_year)
def create_date_table(start_date='2018-01-01', end_date=f'{end_year}-12-31'):
    """
    Creates a Pandas DataFrame representing a full date table.

    Args:
        start_date (str): The start date for the date table (format 'YYYY-MM-DD').
        end_date (str): The end date for the date table (format 'YYYY-MM-DD').

    Returns:
        pd.DataFrame: A DataFrame containing various date-related columns.
    """
    df = pd.DataFrame({"Date": pd.date_range(start_date, end_date)})

    df["Day"] = df.Date.dt.day_name()
    df["DayOfWeek"] = df.Date.dt.dayofweek + 1 # Monday=1, Sunday=7
    df["DayOfMonth"] = df.Date.dt.day
    df["DayOfYear"] = df.Date.dt.dayofyear

    df["Week"] = df.Date.dt.isocalendar().week.astype(int)
    df["WeekOfYear"] = df.Date.dt.isocalendar().week.astype(int) # Same as Week

    df["Month"] = df.Date.dt.month
    df["MonthName"] = df.Date.dt.month_name()
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["YearQuarter"] = df.Date.dt.strftime('%Y-Q%q')
    df["Year-Month"] = df.Date.dt.strftime('%Y-%m')

    df["IsWeekend"] = df.Date.dt.dayofweek.isin([5, 6]) # Saturday=5, Sunday=6
    df["IsLeapYear"] = df.Date.dt.is_leap_year

    return df

spark.sql(f"DROP TABLE IF EXISTS {workspace_name}.{lakehouse_name}.{schema_name}.DimDate")
# Create the date table
date_table_df = create_date_table()
print("DimDate table dropped")
logActivity("DimDate","DimDate table dropped")
# Display the first few rows of the DataFrame
#display(date_table_df.head())
#df["Year-Month"] = df["Year"] + "-" + df["Month"]
#print(df)

spark_df = spark.createDataFrame(date_table_df)
spark_df.write.mode("overwrite").format("delta").saveAsTable(f"{full_lakehouse_name}.DimDate")
print("DimDate table created")
logActivity("DimDate","DimDate table created.")

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 32, Finished, Available, Finished)

DimDate table dropped
DimDate table created


In [22]:

time.sleep(30)
print("Refreshing lakehouse data")
logActivity("Dataset","Refreshing lakehouse data")
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri,
    key_vault_tenant_id=tenant_id,
    key_vault_client_id=client_id,
    key_vault_client_secret=client_secret):

    labs.refresh_sql_endpoint_metadata(workspace=workspace_name,item=lakehouse_name,type='Lakehouse')

time.sleep(30)
fabric.refresh_dataset(workspace=workspace_name, dataset=semantic_model_name)
print("Dataset refresh completed")
logActivity("Dataset","Dataset refresh completed")

StatementMeta(, dc036f3a-815c-4caa-8741-efceb26af839, 37, Finished, Available, Finished)

Refreshing lakehouse data
🟢 The metadata of the SQL endpoint for the 'Fit_Tool' lakehouse within the 'amaser_dp700' workspace has been refreshed for all tables.
Dataset refresh completed
